In [2]:
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta
#03-05-2015 06:00:05
#18-09-2015 05:59:48



df_events = pd.read_csv('D:\Питон\Project X\Data sets\events.csv')

In [3]:
date_start_input = '05-06-2015 00:00:00'
date_start_reform = int(datetime.strptime(date_start_input, '%d-%m-%Y %H:%M:%S').timestamp() * 1000)
date_end_input = '12-06-2015 23:59:00'
date_end_reform = int(datetime.strptime(date_end_input, '%d-%m-%Y %H:%M:%S').timestamp() * 1000)

df_events_cut = df_events[df_events['timestamp'] > date_start_reform]
df_events_cut = df_events_cut[df_events_cut['timestamp'] < date_end_reform]
df_events_cut.reset_index()

,index,timestamp,visitorid,event,itemid,transactionid
0,70430,1433473049725,302687,view,279385,NaN
1,70431,1433473650262,1235993,view,252806,NaN
2,70432,1433469363201,167178,view,330920,NaN
3,70433,1433472523667,939751,view,38965,NaN
4,70434,1433472736442,317318,view,5411,NaN
...,...,...,...,...,...,...
160670,261309,1434142550793,730404,view,359250,NaN
160671,261311,1434142553808,966427,view,273936,NaN
160672,261314,1434142316745,1102202,view,245333,NaN
160673,261316,1434141864544,116393,view,203917,NaN


In [6]:
user_with_tr = df_events_cut.visitorid.where(df_events_cut.transactionid.notna()).dropna().unique()
sort_df = (df_events_cut[df_events_cut.visitorid.isin(user_with_tr)]
           .sort_values('timestamp')
           .reset_index())
sort_df['diff'] = sort_df.groupby('visitorid')['timestamp'].diff(1)
sort_df['shift'] = sort_df.groupby('visitorid')['timestamp'].shift(1)
del sort_df['index']
max_timesatmp = sort_df['timestamp'].max()
session_time = 900000
sort_df['session'] = sort_df['diff'] < session_time
sort_df

,timestamp,visitorid,event,itemid,transactionid,diff,shift,session
0,1433451664702,438441,view,416170,NaN,NaN,NaN,False
1,1433451694867,438441,view,190606,NaN,30165.0,1.433452e+12,True
2,1433451722703,1170743,transaction,164344,11112.0,NaN,NaN,False
3,1433451751879,892355,view,370064,NaN,NaN,NaN,False
4,1433451762383,503857,view,135840,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...
10403,1434142336536,1204946,view,206241,NaN,97727453.0,1.434045e+12,False
10404,1434142461006,531661,view,32971,NaN,1238962.0,1.434141e+12,False
10405,1434142490653,531661,view,102000,NaN,29647.0,1.434142e+12,True
10406,1434142573806,1204946,view,206241,NaN,237270.0,1.434142e+12,True


In [5]:
sort_df.groupby('visitorid').agg({'event': 'size', 'timestamp': 'max'})

,event,timestamp
visitorid,,
3847,4,1433991428443
4241,3,1434045177660
5117,9,1433894752323
11333,24,1433818193372
16158,5,1433733735943
...,...,...
1395774,6,1433742937363
1397020,2,1433773747908
1398811,24,1433912285800
